Runs expecting a directory structure like ./ProjectName/image_type/images
with "images" containing a mix of images classified by the inclusion of "ductal adenocarcinoma" in the name


In [2]:
import fastbook
from fastbook import *

In [3]:
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

In [4]:
path =Path("D:/pytorch/data/TMA_PDAC_Healthy/image_export_256_1x/")
Path.BASE_PATH =path
image_list = (path/'ductal adenocarcinoma').ls()

In [5]:
# if retraining or testing
#learn = load_learner(path/'PDAC_Healthy_resnet34.pkl')

(#0) []

In [10]:
def is_PDAC(x): 
    if "ductal adenocarcinoma" in x: 
        return True 
    else:
        return False
# def is_PDAC(x): return x[0].isupper()

dls = ImageDataLoaders.from_name_func(
    path, bs=24, fnames = get_image_files(path/"images"),
    label_func = is_PDAC, valid_pct=0.2, seed=42,
    batch_tfms = [*aug_transforms(Rotate(), Brightness(max_lighting:float=0.1), Flip(), Contrast(max_lighting=0.1))],
    num_workers=0
)

In [12]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(5)
#learn.fine_tune(20)

epoch,train_loss,valid_loss,error_rate,time
0,0.260390,0.218501,0.085625,11:00
1,0.213488,0.173679,0.072041,05:57
2,0.176785,0.167826,0.065461,05:58
3,0.185476,0.141438,0.057273,05:58
4,0.152558,0.144417,0.058456,05:59


In [15]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,error_rate,time
0,0.134802,0.126443,0.052397,06:03
1,0.146635,0.128938,0.053486,09:58
2,0.162079,0.136220,0.053439,07:10
3,0.162795,0.161605,0.062574,06:02
4,0.147548,0.135624,0.052587,06:03
5,0.159833,0.129176,0.050788,05:59
6,0.137804,0.267285,0.051167,06:00
7,0.130120,0.232418,0.049037,05:59
8,0.129786,0.537047,0.050599,05:59
9,0.143232,0.115079,0.047143,05:59


In [16]:
learn.fine_tune(20)


epoch,train_loss,valid_loss,error_rate,time
0,0.135537,0.728951,0.054622,06:01


epoch,train_loss,valid_loss,error_rate,time
0,0.142586,3.383639,0.061533,07:04
1,0.153090,0.204009,0.063615,07:05
2,0.189137,0.142793,0.057036,06:56
3,0.170951,0.552719,0.065272,06:57
4,0.144403,0.140627,0.057367,06:57
5,0.151014,14.433071,0.058551,07:19
6,0.158193,0.283303,0.063568,07:17
7,0.145597,0.430961,0.058551,07:17
8,0.124938,0.129050,0.050693,07:18
9,0.134565,0.113013,0.045108,07:17


In [17]:
learn.export('PDAC_Healthy_resnet34.pkl')